# Trans2ray baggage xray object detection

## 📦 Imports & Global Configuration

In this section, we import all required libraries 
We also define global constants used throughout the notebook, such as:
- Input image size
- ImageNet normalization parameters

In [ ]:
# ===============================
# Standard Libraries
# ===============================
import os
import json
import math
import random
import shutil
from glob import glob
from pathlib import Path
from typing import Optional

# ===============================
# Numerical & Data Handling
# ===============================
import numpy as np
import pandas as pd

# ===============================
# PyTorch Core
# ===============================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils import clip_grad_norm_

# ===============================
# Computer Vision
# ===============================
import torchvision
import torchvision.transforms.functional as TF
from PIL import Image

# ===============================
# Visualization
# ===============================
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# ===============================
# Utilities
# ===============================
from tqdm import tqdm
from torchinfo import summary
from scipy.optimize import linear_sum_assignment

# ===============================
# Global Constants
# ===============================
IMG_SIZE = 512

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

## 🎯 Reproducibility & Random Seed Control

To ensure **reproducible experiments**, we fix all relevant random seeds across different libraries and hardware backends.

This function:
- Sets the random seed for **Python**, **NumPy**, and **PyTorch**


In [ ]:
def set_seed(seed: int = 42, deterministic: bool = True):
    # python
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    # numpy
    np.random.seed(seed)

    # torch (CPU)
    torch.manual_seed(seed)

    # torch (CUDA)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # torch (MPS - Apple Silicon)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

    # deterministic behavior (slower but reproducible)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    else:
        torch.backends.cudnn.benchmark = True


# 🔹 call once at the very top
set_seed(42)


In [3]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)


g = torch.Generator()
g.manual_seed(42)

## 📂 Dataset Paths & Class Mapping

This cell defines:
- The **root directory** of the DVXray dataset
- Separate paths for **positive** and **negative** samples
- A class-to-index mapping used for model training

Basic assertions are included to ensure that the dataset paths exist before proceeding.

In [ ]:
ROOT = Path("C:/Users/pc/Downloads/dvxray_dataset")

POS_DIR = ROOT / "positive"
NEG_DIR = ROOT / "negative"

CLASS_MAP = {
    'Gun': 0, 'Knife': 1, 'Wrench': 2, 'Pliers': 3, 'Scissors': 4, 'Lighter': 5, 'Battery': 6,
    'Bat': 7, 'Razor_blade': 8, 'Saw_blade': 9, 'Fireworks': 10, 'Hammer': 11,
    'Screwdriver': 12, 'Dart': 13, 'Pressure_vessel': 14
}

assert POS_DIR.exists(), f"missing: {POS_DIR}"
assert NEG_DIR.exists(), f"missing: {NEG_DIR}"
print("OK")


OK


## 🗂️ Dataset Indexing

This cell parses annotation files and builds a unified index for the dataset.

For each sample, it collects:
- Image paths (OL & SD views)
- Annotation metadata
- Object-level labels (if available)

Both **positive** and **negative** folders are indexed and merged into a single list for downstream processing.

In [5]:
def read_json(p):
    with open(p, "r") as f:
        return json.load(f)

def index_folder(folder: Path):
    items = []
    json_paths = sorted(folder.glob("*.json"))
    for jp in json_paths:
        ann = read_json(jp)
        name = ann["name"]

        ol = folder / f"{name}_OL.png"
        sd = folder / f"{name}_SD.png"

        objs = ann.get("objects", [])
        if objs == "None" or objs is None:
            objs = []

        items.append({
            "name": name,
            "folder": folder.name,   # positive/negative
            "ol_path": str(ol),
            "sd_path": str(sd),
            "json_path": str(jp),
            "objects": objs
        })
    return items

items = index_folder(POS_DIR) + index_folder(NEG_DIR)
print("Total samples:", len(items))


Total samples: 16000


## 🔍 Dataset Integrity Check

This cell performs basic sanity checks on the indexed dataset, including:
- Counting **positive** and **negative** samples
- Verifying the existence of **OL** and **SD** image files
- Validating object labels against the predefined class map

Summary statistics and example anomalies are printed to quickly identify potential data issues before training.

In [ ]:
missing_ol = []
missing_sd = []
bad_json = []
unknown_labels = []

pos_count = 0
neg_count = 0
obj_count = 0

for it in items:
    if it["folder"] == "positive":
        pos_count += 1
    else:
        neg_count += 1

    if not os.path.exists(it["ol_path"]):
        missing_ol.append(it["name"])
    if not os.path.exists(it["sd_path"]):
        missing_sd.append(it["name"])

    # label check
    for o in it["objects"]:
        obj_count += 1
        lab = o.get("label")
        if lab not in CLASS_MAP:
            unknown_labels.append((it["name"], lab))

print("positive:", pos_count, "negative:", neg_count)
print("total objects:", obj_count)
print("missing OL:", len(missing_ol), "missing SD:", len(missing_sd))
print("unknown labels:", len(unknown_labels))

if missing_ol[:3]: print("missing_ol ex:", missing_ol[:3])
if missing_sd[:3]: print("missing_sd ex:", missing_sd[:3])
if unknown_labels[:3]: print("unknown_labels ex:", unknown_labels[:3])


positive: 5000 negative: 11000
total objects: 5087
missing OL: 0 missing SD: 0
unknown labels: 0


## 🧹 Annotation Cleaning & Bounding Box Validation

This cell validates and cleans annotation files by removing invalid bounding boxes.

The cleaning process:
- Checks bounding boxes for valid `(x1, y1, x2, y2)` format
- Removes degenerate, zero-area, or negative-coordinate boxes
- Ensures both **OL** and **SD** views are valid for each object
- Creates a backup of original annotation files before modification

A summary of modified files and removed objects is printed separately for positive and negative samples.

In [ ]:
ROOT = Path("C:/Users/pc/Downloads/dvxray_dataset")
POS_DIR = ROOT / "positive"
NEG_DIR = ROOT / "negative"

def valid_xyxy(bb):
    if not isinstance(bb, list) or len(bb) != 4:
        return False
    x1,y1,x2,y2 = bb
    # all zeros or degenerate
    if (x1,y1,x2,y2) == (0,0,0,0):
        return False
    if x2 <= x1 or y2 <= y1:
        return False
    if x1 < 0 or y1 < 0:
        return False
    return True

def clean_json_file(js_path: Path, make_backup=True):
    with open(js_path, "r") as f:
        ann = json.load(f)

    objs = ann.get("objects", [])
    if objs == "None" or objs is None:
        return 0, 0  # unchanged, removed=0

    kept = []
    removed = 0
    for o in objs:
        ol_ok = valid_xyxy(o.get("ol_bb"))
        sd_ok = valid_xyxy(o.get("sd_bb"))
        if ol_ok and sd_ok:
            kept.append(o)
        else:
            removed += 1

    if removed == 0:
        return 0, 0  # nothing removed

    # backup
    if make_backup:
        bkp = js_path.with_suffix(".json.bak")
        if not bkp.exists():
            shutil.copy2(js_path, bkp)

    # write cleaned
    if len(kept) == 0:
        ann["objects"] = "None"
    else:
        ann["objects"] = kept

    with open(js_path, "w") as f:
        json.dump(ann, f)

    return removed, len(kept)

def clean_folder(folder: Path):
    removed_total = 0
    touched = 0
    for js_path in folder.glob("*.json"):
        removed, kept = clean_json_file(js_path)
        if removed > 0:
            touched += 1
            removed_total += removed
    return touched, removed_total

t_pos, r_pos = clean_folder(POS_DIR)
t_neg, r_neg = clean_folder(NEG_DIR)

print("Touched JSON files (pos):", t_pos, "Removed objects:", r_pos)
print("Touched JSON files (neg):", t_neg, "Removed objects:", r_neg)


Touched JSON files (pos): 0 Removed objects: 0
Touched JSON files (neg): 0 Removed objects: 0


## 📐 Bounding Box Consistency Check

This cell performs an additional validation pass over all bounding boxes to detect:
- Incorrect formats or missing values
- Non-numeric coordinates
- Degenerate boxes with zero or negative area
- Bounding boxes with negative coordinates

Detected anomalies are counted and a few examples are printed for manual inspection.

In [ ]:
bad_bbox = []
for it in items:
    for o in it["objects"]:
        for key in ["ol_bb", "sd_bb"]:
            bb = o.get(key)
            if not isinstance(bb, list) or len(bb) != 4:
                bad_bbox.append((it["name"], key, bb))
                continue
            x1,y1,x2,y2 = bb
            if not all(isinstance(v, (int, float)) for v in bb):
                bad_bbox.append((it["name"], key, bb))
                continue
            if x2 <= x1 or y2 <= y1:
                bad_bbox.append((it["name"], key, bb))
            if x1 < 0 or y1 < 0:
                bad_bbox.append((it["name"], key, bb))
ط
print("bad_bbox:", len(bad_bbox))
if bad_bbox[:5]:
    print("examples:", bad_bbox[:5])


bad_bbox: 0


## 🧠 Two-View DVXray Dataset for DETR

This section defines a custom PyTorch `Dataset` for the **DVXray two-view object detection task**, designed to be compatible with **DETR-style models**.

Key features:
- Supports **two synchronized views** per sample (OL & SD)
- Loads annotations from JSON files
- Cleans and filters invalid objects and labels
- Resizes images and scales bounding boxes accordingly
- Converts bounding boxes from `xyxy` format to **normalized `cxcywh`**
- Applies **ImageNet normalization**

The dataset outputs:
- Two normalized image tensors (OL, SD)
- A target dictionary containing labels and bounding boxes for both views

In [ ]:
# ---------- Configuration ----------
CLASS_MAP = {
    'Gun': 0, 'Knife': 1, 'Wrench': 2, 'Pliers': 3, 'Scissors': 4, 'Lighter': 5, 'Battery': 6,
    'Bat': 7, 'Razor_blade': 8, 'Saw_blade': 9, 'Fireworks': 10, 'Hammer': 11,
    'Screwdriver': 12, 'Dart': 13, 'Pressure_vessel': 14
}

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]


# ---------- Utility Functions ----------
def xyxy_to_cxcywh_norm(boxes_xyxy: torch.Tensor, W: int, H: int) -> torch.Tensor:
    """
    Convert absolute bounding boxes from (x1, y1, x2, y2) format
    to normalized (cx, cy, w, h) format in range [0, 1].
    """
    if boxes_xyxy.numel() == 0:
        return boxes_xyxy.reshape(0, 4)

    x1, y1, x2, y2 = boxes_xyxy.unbind(-1)
    cx = (x1 + x2) / 2.0
    cy = (y1 + y2) / 2.0
    bw = (x2 - x1)
    bh = (y2 - y1)

    out = torch.stack([cx / W, cy / H, bw / W, bh / H], dim=-1)
    return out.clamp(0.0, 1.0)


def resize_img_and_boxes_xyxy(img: Image.Image, boxes_xyxy: torch.Tensor, size: int):
    """
    Resize image to (size, size) and scale bounding boxes accordingly.
    """
    w0, h0 = img.size
    img_resized = img.resize((size, size), resample=Image.BILINEAR)

    if boxes_xyxy.numel() == 0:
        return img_resized, boxes_xyxy.reshape(0, 4)

    sx = size / w0
    sy = size / h0

    boxes = boxes_xyxy.clone()
    boxes[:, [0, 2]] *= sx
    boxes[:, [1, 3]] *= sy

    boxes[:, [0, 2]] = boxes[:, [0, 2]].clamp(0, size - 1)
    boxes[:, [1, 3]] = boxes[:, [1, 3]].clamp(0, size - 1)

    return img_resized, boxes


# ---------- Dataset ----------
class DVXrayTwoViewDETR(Dataset):
    """
    DVXray two-view dataset for DETR-style object detection.

    Expected structure:
    root/
      Positive/*.json + *_OL.png + *_SD.png
      Negative/*.json + *_OL.png + *_SD.png
    """

    def __init__(self, root_dir: str, split: str = "all", img_size: int = 512):
        self.root = Path(root_dir)
        self.img_size = img_size

        # If no explicit split folders exist, use the root directory
        base_dir = self.root if split == "all" else (self.root / split)

        pos_dir = base_dir / "Positive"
        neg_dir = base_dir / "Negative"

        assert pos_dir.exists(), f"Missing folder: {pos_dir}"
        assert neg_dir.exists(), f"Missing folder: {neg_dir}"

        self.samples = []

        for folder in [pos_dir, neg_dir]:
            for js_path in sorted(folder.glob("*.json")):
                ann = json.loads(js_path.read_text())
                name = ann["name"]

                ol_path = folder / f"{name}_OL.png"
                sd_path = folder / f"{name}_SD.png"

                if not ol_path.exists() or not sd_path.exists():
                    continue

                objects = ann.get("objects", [])
                if objects == "None" or objects is None:
                    objects = []

                self.samples.append((ol_path, sd_path, objects, name))

        if len(self.samples) == 0:
            raise RuntimeError(
                "No samples found. Check folder names (Positive/Negative) and file naming (*_OL.png, *_SD.png)."
            )

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        ol_path, sd_path, objects, name = self.samples[idx]

        # Load images
        img_ol = Image.open(ol_path).convert("RGB")
        img_sd = Image.open(sd_path).convert("RGB")

        labels = []
        boxes_ol = []
        boxes_sd = []

        # Parse annotations
        for obj in objects:
            label_name = obj["label"]
            if label_name not in CLASS_MAP:
                continue

            labels.append(CLASS_MAP[label_name])
            boxes_ol.append(obj["ol_bb"])  # xyxy
            boxes_sd.append(obj["sd_bb"])  # xyxy

        labels = torch.tensor(labels, dtype=torch.int64)
        boxes_ol = torch.tensor(boxes_ol, dtype=torch.float32).reshape(-1, 4)
        boxes_sd = torch.tensor(boxes_sd, dtype=torch.float32).reshape(-1, 4)

        # Resize images and scale bounding boxes (each view independently)
        img_ol, boxes_ol = resize_img_and_boxes_xyxy(img_ol, boxes_ol, self.img_size)
        img_sd, boxes_sd = resize_img_and_boxes_xyxy(img_sd, boxes_sd, self.img_size)

        # Convert images to tensor and normalize
        x_ol = TF.normalize(TF.to_tensor(img_ol), IMAGENET_MEAN, IMAGENET_STD)
        x_sd = TF.normalize(TF.to_tensor(img_sd), IMAGENET_MEAN, IMAGENET_STD)

        # Convert boxes to DETR-style normalized cxcywh format
        boxes_ol_n = xyxy_to_cxcywh_norm(boxes_ol, self.img_size, self.img_size)
        boxes_sd_n = xyxy_to_cxcywh_norm(boxes_sd, self.img_size, self.img_size)

        target = {
            "image_id": torch.tensor([idx]),
            "name": name,
            "labels": labels,        # class indices [0..14]
            "boxes_ol": boxes_ol_n,  # [N, 4] normalized cxcywh
            "boxes_sd": boxes_sd_n,  # [N, 4] normalized cxcywh
        }

        return x_ol, x_sd, target

## 📦 Custom Collate Function (Two-View DETR)

This function defines a custom `collate_fn` for batching two-view inputs (OL & SD) while keeping detection targets as a list, as required by **DETR-style models**.

In [12]:
def collate_two_view_detr(batch):
    x_ol, x_sd, targets = zip(*batch)
    return torch.stack(x_ol, 0), torch.stack(x_sd, 0), list(targets)

## 🚦 Dataset Split & DataLoaders

Split the DVXray dataset into train/val/test and create DataLoaders with:
- Two-view batching (OL & SD)
- Custom collate function
- Reproducible seeding

In [ ]:

g = torch.Generator()
g.manual_seed(42)   

full_dataset = DVXrayTwoViewDETR(
    root_dir=ROOT,
    split="all",
    img_size=512
)
n = len(full_dataset)
n_train = int(0.8 * n)
n_val   = int(0.1 * n)
n_test  = n - n_train - n_val
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset,
    [n_train, n_val, n_test],
    generator=g
)

dl_train = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_two_view_detr,
    worker_init_fn=seed_worker,
    generator=g        
)

dl_val = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_two_view_detr,
    worker_init_fn=seed_worker,
    generator=g
)

dl_test = DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=collate_two_view_detr,
    worker_init_fn=seed_worker,
    generator=g
)


## 🏗️ CVFF Transformer Encoder (GCA + LFS)

This module implements a **custom Transformer encoder** for two-view feature fusion in DVXray, inspired by Trans2Ray:

Key features:
- **Global Cross Attention (GCA)**: Cross attention between main and assist views, applied multiple times.
- **Local Feature Selection (LFS)**: Encoder layers with block + MAWS for selecting top-K informative tokens.
- **Two-view token fusion**: Combines main and assist features with global pooled tokens.
- Output: `[N, B, C]` fused features for main view, ready for DETR-style decoder.

In [ ]:
# -----------------------------
# MAWS: Mutual Attention Weight Selection
# -----------------------------
class MAWS(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, contributions):
        # x: attention [B, H, N, N] from a Block
        # contributions: [B, H, N] or similar
        contributions = contributions.mean(1)          # [B, N], mean over heads
        weights = x[:, :, 0, :].mean(1)                # [B, N], mean over heads at query=0
        scores = contributions * weights               # [B, N]
        max_inx = torch.argsort(scores, dim=1, descending=True)
        return None, max_inx


# -----------------------------
# Attention + MLP + Block
# -----------------------------
ACT2FN = {"gelu": torch.nn.functional.gelu}

class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.wq = nn.Linear(dim, dim, bias=qkv_bias)
        self.wk = nn.Linear(dim, dim, bias=qkv_bias)
        self.wv = nn.Linear(dim, dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        # x: [N, B, C] -> [B, N, C] for attention computation
        x = x.view(x.size(1), x.size(0), x.size(2))
        B, N, C = x.shape

        q = self.wq(x).reshape(B, N, self.num_heads, C//self.num_heads).permute(0,2,1,3)
        k = self.wk(x).reshape(B, N, self.num_heads, C//self.num_heads).permute(0,2,1,3)
        v = self.wv(x).reshape(B, N, self.num_heads, C//self.num_heads).permute(0,2,1,3)

        attn_scores = (q @ k.transpose(-2, -1)) * self.scale  # [B,H,N,N]
        attn = attn_scores.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = (attn @ v).transpose(1,2).reshape(B, N, C)
        out = self.proj(out)
        out = self.proj_drop(out)

        out = out.view(out.size(1), out.size(0), out.size(2))  # back to [N,B,C]

        # contribution = softmax over keys for query=0
        contribution = attn_scores.softmax(dim=-2)[:,:,:,0]  # [B,H,N]
        return out, attn, contribution


class Mlp(nn.Module):
    def __init__(self, dim, r=4):
        super().__init__()
        self.fc1 = nn.Linear(dim, r*dim)
        self.fc2 = nn.Linear(r*dim, dim)
        self.act_fn = ACT2FN["gelu"]
        self.dropout = nn.Dropout()

        nn.init.xavier_uniform_(self.fc1.weight); nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.normal_(self.fc1.bias, std=1e-6); nn.init.normal_(self.fc2.bias, std=1e-6)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act_fn(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class Block(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.attention_norm = nn.LayerNorm(dim, eps=1e-6)
        self.ffn_norm = nn.LayerNorm(dim, eps=1e-6)
        self.ffn = Mlp(dim)
        self.attn = Attention(dim)

    def forward(self, x):
        # x: [N, B, C]
        h = x
        x = self.attention_norm(x)
        x, weights, contribution = self.attn(x)
        x = x + h

        h = x
        x = self.ffn_norm(x)
        x = self.ffn(x)
        x = x + h
        return x, weights, contribution


# -----------------------------
# CrossAttention + CrossBlock
# -----------------------------
class CrossAttention(nn.Module):
    def __init__(self, dim, num_heads, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.wq = nn.Linear(dim, dim, bias=qkv_bias)
        self.wk = nn.Linear(dim, dim, bias=qkv_bias)
        self.wv = nn.Linear(dim, dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, d):
        # x: [N2, B, C] -> [B, N2, C]
        x = x.view(x.size(1), x.size(0), x.size(2))
        B, N, C = x.shape

        # query from first d tokens (main)
        q = self.wq(x[:, :d, :]).reshape(B, d, self.num_heads, C//self.num_heads).permute(0,2,1,3)
        k = self.wk(x).reshape(B, N, self.num_heads, C//self.num_heads).permute(0,2,1,3)
        v = self.wv(x).reshape(B, N, self.num_heads, C//self.num_heads).permute(0,2,1,3)

        attn = (q @ k.transpose(-2, -1)) * self.scale  # [B,H,d,N]
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        out = (attn @ v).transpose(1,2).reshape(B, d, C)
        out = self.proj(out)
        out = self.proj_drop(out)

        out = out.view(out.size(1), out.size(0), out.size(2))  # [d,B,C]
        return out, attn


class CrossBlock(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super().__init__()
        self.self_attn = CrossAttention(d_model, num_heads=nhead, attn_drop=0.1, proj_drop=0.1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, d):
        # src: [2N, B, C], d=N
        s, attn = self.self_attn(src, d)
        # only update main tokens
        src_main = src[:d, :, :] + self.dropout(s)
        return src_main, attn[:, :, :, d:]


# -----------------------------
# CVFF Transformer Encoder
# -----------------------------
class CVFFTransformerEncoder(nn.Module):
    """
    Input: src [N, 2B, C] (main + assist)
    Output: memory_main [N, B, C]
    Implements:
      - GCA (2x CrossBlock)
      - LFS with top-K token selection (MAWS)
      - Optional feature fusion
    """
    def __init__(self, d_model=256, nhead=8, num_layers=6,
                 dim_feedforward=1024, dropout=0.1,
                 gca_iters=2, topk=8, feature_fusion=True, use_norm=True):
        super().__init__()
        self.num_layers = num_layers
        self.feature_fusion = feature_fusion
        self.gca_iters = gca_iters
        self.topk = topk

        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model) if use_norm else None
        self.crossblock = CrossBlock(d_model, nhead=nhead, dropout=dropout)
        self.globle_pool = nn.AdaptiveAvgPool1d(1)
        self.maws = MAWS()
        self.block = Block(d_model)

    def forward(self, src, mask=None, src_key_padding_mask=None, pos=None):
        # src: [N, 2B, C]
        N, B2, C = src.shape
        assert B2 % 2 == 0, "Batch must be even: main+assist"
        B = B2 // 2

        output = src

        # ---- GCA ----
        for _ in range(self.gca_iters):
            output_main = output[:, :B, :]
            output_assist = output[:, B:, :]
            new_tokens = torch.cat((output_main, output_assist), dim=0)  # [2N,B,C]
            output_main, _ = self.crossblock(new_tokens, N)
            output = torch.cat((output_main, output_assist), dim=1)

        # ---- LFS ----
        tokens = [[] for _ in range(B)]
        count = 0
        for layer in self.layers:
            output = layer(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask, pos=pos)
            count += 1

            output_main = output[:, :B, :]
            output_assist = output[:, B:, :]

            # global pooled token from main: [1,B,C]
            output_m = self.globle_pool(output_main.permute(1,2,0)).permute(2,0,1)
            new_output = torch.cat((output_m, output_assist), dim=0)  # [N+1,B,C]

            # Block + MAWS
            _, weights, contribution = self.block(new_output)

            if self.feature_fusion:
                _, selected_inx = self.maws(weights, contribution)
                new_output_bnc = new_output.permute(1,0,2)  # [B,N+1,C]
                K = min(self.topk, new_output_bnc.shape[1])
                for i in range(B):
                    tokens[i].extend(new_output_bnc[i, selected_inx[i,:K]])

                # maintain main+assist for next layer except last
                if count < self.num_layers:
                    output = torch.cat((output_main, output_assist), dim=1)
                else:
                    output = output_main

        # mean selected tokens
        fused = []
        for i in range(B):
            if len(tokens[i]) == 0:
                fused.append(torch.zeros(C, device=output.device, dtype=output.dtype))
            else:
                fused.append(torch.stack(tokens[i], dim=0).mean(0))
        fused = torch.stack(fused, dim=0).unsqueeze(0)  # [1,B,C]

        # final fusion
        output_main = output[:, :B, :] if output.shape[1]==2*B else output
        output = output_main + fused

        if self.norm is not None:
            output = self.norm(output)

        return output  # [N,B,C]

## 📦 Bounding Box Utilities

This cell provides helper functions for bounding box operations:

- **Conversions**:
  - `box_cxcywh_to_xyxy`: center-format → corner-format
  - `box_xyxy_to_cxcywh`: corner-format → center-format
- **Area computation**:
  - `box_area`: computes area of boxes
- **Generalized IoU**:
  - `generalized_box_iou`: computes [N,M] GIoU between two sets of xyxy boxes

In [ ]:
def box_cxcywh_to_xyxy(x):
    # x: [...,4] in cxcywh normalized or absolute
    cx, cy, w, h = x.unbind(-1)
    b = [(cx - 0.5*w), (cy - 0.5*h),
         (cx + 0.5*w), (cy + 0.5*h)]
    return torch.stack(b, dim=-1)

def box_xyxy_to_cxcywh(x):
    x0, y0, x1, y1 = x.unbind(-1)
    b = [(x0 + x1)/2, (y0 + y1)/2, (x1 - x0), (y1 - y0)]
    return torch.stack(b, dim=-1)

def box_area(boxes):
    return (boxes[..., 2] - boxes[..., 0]).clamp(min=0) * (boxes[..., 3] - boxes[..., 1]).clamp(min=0)

def generalized_box_iou(boxes1, boxes2):
    """
    boxes1: [N,4] xyxy, boxes2: [M,4] xyxy
    returns: [N,M]
    """
    # intersection
    lt = torch.max(boxes1[:, None, :2], boxes2[None, :, :2])
    rb = torch.min(boxes1[:, None, 2:], boxes2[None, :, 2:])
    wh = (rb - lt).clamp(min=0)
    inter = wh[:, :, 0] * wh[:, :, 1]

    area1 = box_area(boxes1)[:, None]
    area2 = box_area(boxes2)[None, :]
    union = area1 + area2 - inter + 1e-9
    iou = inter / union

    # enclosing box
    lt_c = torch.min(boxes1[:, None, :2], boxes2[None, :, :2])
    rb_c = torch.max(boxes1[:, None, 2:], boxes2[None, :, 2:])
    wh_c = (rb_c - lt_c).clamp(min=0)
    area_c = wh_c[:, :, 0] * wh_c[:, :, 1] + 1e-9

    giou = iou - (area_c - union) / area_c
    return giou


## 🧭 DETR-style 2D Sine-Cosine Positional Encoding

`PositionEmbeddingSine` generates 2D sine-cosine positional encodings for transformer inputs.

- **Input**: `[B, C, H, W]` feature map  
- **Output**: `[B, 2*num_pos_feats, H, W]` positional encoding  
- **Parameters**:
  - `num_pos_feats`: number of features per dimension (default 128)
  - `temperature`: frequency scaling factor (default 10000)
  - `normalize`: whether to normalize positions to [0, scale]  
  - `scale`: optional scaling factor (default `2*pi`)
- **Logic**:  
  - Compute normalized x/y coordinates  
  - Apply sine to even channels, cosine to odd channels  
  - Concatenate x/y embeddings along feature dimension

In [ ]:
class PositionEmbeddingSine(nn.Module):
    """
    DETR-style 2D sine-cosine positional encoding.
     [B, 2*num_pos_feats, H, W]
    """
    def __init__(self, num_pos_feats=128, temperature=10000, normalize=True, scale=None):
        super().__init__()
        self.num_pos_feats = num_pos_feats
        self.temperature = temperature
        self.normalize = normalize
        if scale is None:
            scale = 2 * math.pi
        self.scale = scale

    def forward(self, x):
        # x: [B,C,H,W]
        B, _, H, W = x.shape
        device = x.device
        dtype = x.dtype

        y_embed = torch.arange(H, device=device, dtype=dtype).unsqueeze(1).repeat(1, W)
        x_embed = torch.arange(W, device=device, dtype=dtype).unsqueeze(0).repeat(H, 1)

        if self.normalize:
            y_embed = y_embed / (H - 1 + 1e-6) * self.scale
            x_embed = x_embed / (W - 1 + 1e-6) * self.scale

        dim_t = torch.arange(self.num_pos_feats, device=device, dtype=dtype)
        dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)

        # [H,W,1] / [F] => [H,W,F]
        pos_x = x_embed[..., None] / dim_t
        pos_y = y_embed[..., None] / dim_t

        
        pos_x = torch.stack((pos_x[..., 0::2].sin(), pos_x[..., 1::2].cos()), dim=-1).flatten(-2)  # [H,W,F]
        pos_y = torch.stack((pos_y[..., 0::2].sin(), pos_y[..., 1::2].cos()), dim=-1).flatten(-2)  # [H,W,F]

        pos = torch.cat((pos_y, pos_x), dim=-1)               # [H,W,2F]
        pos = pos.permute(2, 0, 1).unsqueeze(0).repeat(B, 1, 1, 1)  # [B,2F,H,W]
        return pos


## 🏗 ResNet Backbone with Frozen BatchNorm

This cell defines a **ResNet18 backbone** for feature extraction with frozen batch normalization.

- **FrozenBatchNorm2d**: BatchNorm layers with fixed parameters (no running stats update).  
- **ResNetBackbone**:
  - Loads ResNet18 architecture.
  - Replaces standard BatchNorm2d with `FrozenBatchNorm2d`.
  - Projects the last feature map to `out_channels` (default 256).
  - Freezes all layers except `layer4` and the projection layer.
  - Optionally loads pretrained weights from a local file.

- **Forward Pass**:  
  1. Stem: conv1 → bn1 → relu → maxpool  
  2. Residual layers: layer1 → layer2 → layer3 → layer4  
  3. Projection to desired channel dimension

In [ ]:
# -----------------------------
# Frozen BatchNorm
# -----------------------------
class FrozenBatchNorm2d(nn.Module):
    def __init__(self, num_features):
        super(FrozenBatchNorm2d, self).__init__()
        # Buffers to store weights, bias, running mean/var (not trainable)
        self.register_buffer("weight", torch.ones(num_features))
        self.register_buffer("bias", torch.zeros(num_features))
        self.register_buffer("running_mean", torch.zeros(num_features))
        self.register_buffer("running_var", torch.ones(num_features))

    def forward(self, x):
        scale = self.weight / (self.running_var + 1e-5).sqrt()
        return x * scale + self.bias


# -----------------------------
# ResNet Backbone
# -----------------------------
class ResNetBackbone(nn.Module):
    def __init__(self, out_channels=256, pretrained=False, model_weights_path=None):
        super().__init__()

        # Load ResNet18 architecture without pretrained weights
        m = torchvision.models.resnet18(weights=None)

        # Stem: conv1 → bn1 → relu → maxpool
        self.stem = nn.Sequential(
            m.conv1, 
            m.bn1,  # standard BatchNorm
            m.relu, 
            m.maxpool
        )
        self.layer1 = m.layer1
        self.layer2 = m.layer2
        self.layer3 = m.layer3
        self.layer4 = m.layer4  # last layer remains trainable

        # Project final channels to out_channels
        self.proj = nn.Conv2d(512, out_channels, kernel_size=1)

        # Freeze all layers except layer4 and projection
        self.freeze_layers()

        # Load pretrained weights from file if provided
        if model_weights_path:
            self.load_weights(model_weights_path)

    def freeze_layers(self):
        # Freeze all parameters except layer4 and proj
        for name, param in self.named_parameters():
            if "layer4" not in name and "proj" not in name:
                param.requires_grad = False

    def load_weights(self, model_weights_path):
        """
        Load pretrained weights from local file.
        Only layers present in model dict are loaded.
        """
        print(f"Loading weights from {model_weights_path}")
        checkpoint = torch.load(model_weights_path)
        model_dict = self.state_dict()
        pretrained_dict = {k: v for k, v in checkpoint.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.load_state_dict(model_dict)
        print("Weights loaded successfully.")

    def forward(self, x):
        # Pass through stem and residual layers
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)      # [B,512,H',W']
        x = self.proj(x)        # [B,256,H',W']
        return x


# Example usage
model_weights_path = "C:/Users/pc/Desktop/xray project/resnet18-5c106cde.pth"
model = ResNetBackbone(out_channels=256, pretrained=False, model_weights_path=model_weights_path)
print(model)

Loading weights from C:/Users/pc/Desktop/xray project/resnet18-5c106cde.pth


C:\Users\pc\AppData\Local\Temp\ipykernel_6972\263561293.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_weights_path)


Weights loaded successfully.
ResNetBackbone(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

## 🧩 MLP & DETR Decoder

This cell defines the **MLP head** and a **Transformer-based DETR decoder**:

### `MLP`
- Fully connected feedforward network.
- `num_layers` linear layers with ReLU activations except for the last layer.
- Input → Hidden → … → Output dimensions configurable.

### `DETRDecoder`
- Standard Transformer decoder (PyTorch `nn.TransformerDecoder`).
- Input:
  - `tgt`: target queries `[num_queries, B, C]`
  - `memory`: encoder memory `[N, B, C]`
  - `pos` / `query_pos`: optional positional encodings
- Output: decoded feature `[num_queries, B, C]` normalized via LayerNorm.

In [20]:
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super().__init__()
        layers = []
        for i in range(num_layers):
            in_d = input_dim if i == 0 else hidden_dim
            out_d = output_dim if i == num_layers - 1 else hidden_dim
            layers.append(nn.Linear(in_d, out_d))
        self.layers = nn.ModuleList(layers)

    def forward(self, x):
        for i, l in enumerate(self.layers):
            x = l(x)
            if i < len(self.layers) - 1:
                x = F.relu(x)
        return x

class DETRDecoder(nn.Module):
    def __init__(self, d_model=256, nhead=8, num_layers=6, dim_feedforward=1024, dropout=0.1):
        super().__init__()
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=False
        )
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, memory, pos=None, query_pos=None):
        # tgt: [num_queries,B,C]
        # memory: [N,B,C]
        q = tgt if query_pos is None else tgt + query_pos
        k = memory if pos is None else memory + pos
        hs = self.decoder(q, k)     # [num_queries,B,C]
        hs = self.norm(hs)
        return hs


## 🚀 Trans2Ray Model

This cell implements the **full Trans2Ray detection model**:

### Architecture:
1. **Backbone**: `ResNetBackbone` extracts features from both OL and SD views.
2. **Positional Encoding**: `PositionEmbeddingSine` applied to OL features.
3. **CVFF Encoder**: `CVFFTransformerEncoder` performs:
   - GCA (Global Cross Attention)
   - LFS (Layer-wise Feature Selection)
   - MAWS + Block fusion
4. **DETR Decoder**: Standard transformer decoder for object queries.
5. **Heads**:
   - `class_embed`: predicts class probabilities (+1 for no-object)
   - `bbox_embed`: predicts normalized bounding boxes in cxcywh format.

### Forward Pass:
- Input: `x_ol`, `x_sd` → feature extraction via backbone.
- Flatten spatial dims → `[N, B, C]` for both views.
- Concatenate main + assist features → feed to CVFF encoder.
- Decoder uses learned query embeddings to attend encoder memory.
- Output: 
  - `"pred_logits"` → class predictions `[B, Q, num_classes+1]`
  - `"pred_boxes"` → bounding boxes `[B, Q, 4]` (normalized cxcywh)

In [ ]:
class Trans2RayModel(nn.Module):
    def __init__(self, num_classes=15, num_queries=100, d_model=256,
                 nhead=8, enc_layers=6, dec_layers=6):
        super().__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        self.d_model = d_model

        self.backbone = ResNetBackbone(out_channels=d_model)
        self.pos_enc = PositionEmbeddingSine(num_pos_feats=d_model//2)  # => 256

        # EXACT CVFF encoder (GCA+LFS+MAWS+Block)
        self.encoder = CVFFTransformerEncoder(
            d_model=d_model, nhead=nhead, num_layers=enc_layers,
            dim_feedforward=1024, dropout=0.1, gca_iters=2, topk=8,
            feature_fusion=True, use_norm=True
        )
        self.decoder = DETRDecoder(d_model=d_model, nhead=nhead, num_layers=dec_layers, dim_feedforward=1024)

        self.query_embed = nn.Embedding(num_queries, d_model)
        self.class_embed = nn.Linear(d_model, num_classes + 1)  
        self.bbox_embed = MLP(d_model, d_model, 4, 3)

    def forward(self, x_ol, x_sd):
        # backbone
        f_ol = self.backbone(x_ol)    # [B,256,H',W']
        f_sd = self.backbone(x_sd)    # [B,256,H',W']

        # positional encoding
        pos = self.pos_enc(f_ol)      
        B, C, H, W = f_ol.shape

        # flatten -> [N,B,C]
        N = H * W
        src_main = f_ol.flatten(2).permute(2, 0, 1)  # [N,B,C]
        src_asst = f_sd.flatten(2).permute(2, 0, 1)  # [N,B,C]
        pos_flat = pos.flatten(2).permute(2, 0, 1)      # [N,B,C]

      
        pos_cat = torch.cat([pos_flat, pos_flat], dim=1)  # [N,2B,C]

        src = torch.cat([src_main, src_asst], dim=1)      # [N,2B,C]
        memory = self.encoder(src, pos=pos_cat)           # ✅


        # decoder
        query_pos = self.query_embed.weight.unsqueeze(1).repeat(1, B, 1)  # [Q,B,C]
        tgt = torch.zeros_like(query_pos)                                 # [Q,B,C]
        hs = self.decoder(tgt, memory, pos=pos_flat, query_pos=query_pos)

        # predictions
        outputs_class = self.class_embed(hs).permute(1, 0, 2)  # [B,Q,K+1]
        outputs_coord = self.bbox_embed(hs).sigmoid().permute(1, 0, 2)  # [B,Q,4] normalized cxcywh

        return {"pred_logits": outputs_class, "pred_boxes": outputs_coord}


## 📊 Model Summary

- Set device (`cuda` if available, else `cpu`).
- Instantiate `Trans2RayModel` and move to device.
- Use `torchinfo.summary` to inspect model:
  - Inputs: OL & SD images `[1,3,512,512]`
  - Columns: input size, output size, number of parameters, trainable status.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Trans2RayModel(num_classes=15, num_queries=100, d_model=256, nhead=8, enc_layers=6, dec_layers=6).to(device)
model = model.to(device)

summary(
    model,
    input_size=[(1, 3, 512, 512), (1, 3, 512, 512)],  # x_ol , x_sd
    depth=4,
    col_names=[
        "input_size",
        "output_size",
        "num_params",
        "trainable"
    ],
    device=device
)


Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
Trans2RayModel                                          [1, 3, 512, 512]          [1, 100, 4]               25,600                    Partial
├─ResNetBackbone: 1-1                                   [1, 3, 512, 512]          [1, 256, 16, 16]          --                        Partial
│    └─Sequential: 2-1                                  [1, 3, 512, 512]          [1, 64, 128, 128]         --                        False
│    │    └─Conv2d: 3-1                                 [1, 3, 512, 512]          [1, 64, 256, 256]         (9,408)                   False
│    │    └─BatchNorm2d: 3-2                            [1, 64, 256, 256]         [1, 64, 256, 256]         (128)                     False
│    │    └─ReLU: 3-3                                   [1, 64, 256, 256]         [1, 64, 256, 256]         --                        --
│    │    └─Max

## 🧩 Hungarian Matcher & SetCriterion

- **HungarianMatcher**:
  - Matches predicted boxes and classes to targets using the Hungarian algorithm.
  - Computes a cost matrix combining:
    - Classification cost (`-P(class)`)
    - L1 distance for bounding boxes
    - Generalized IoU
  - Returns indices of matched predictions and targets.

- **SetCriterion**:
  - Computes losses for DETR-style predictions.
  - Losses:
    - `loss_labels`: cross-entropy over predicted classes with "no-object" weighting.
    - `loss_boxes`: L1 loss for bounding boxes + GIoU loss.
  - `forward` matches predictions to targets and sums weighted losses.

In [ ]:
class HungarianMatcher(nn.Module):
    def __init__(self, cost_class=1.0, cost_bbox=5.0, cost_giou=2.0):
        super().__init__()
        # Set weights for each type of cost
        self.cost_class = cost_class
        self.cost_bbox = cost_bbox
        self.cost_giou = cost_giou
        assert cost_class != 0 or cost_bbox != 0 or cost_giou != 0

    @torch.no_grad()
    def forward(self, outputs, targets):
        """
        Match predicted boxes to targets using Hungarian algorithm.
        outputs: dict with pred_logits [B,Q,K+1], pred_boxes [B,Q,4]
        targets: list of dicts, each with labels [N], boxes_ol [N,4]
        """
        bs, num_queries = outputs["pred_logits"].shape[:2]
        out_prob = outputs["pred_logits"].softmax(-1)  # classification probabilities
        out_bbox = outputs["pred_boxes"]               # predicted boxes

        indices = []
        for b in range(bs):
            tgt_ids = targets[b]["labels"].to(out_bbox.device)
            tgt_bbox = targets[b]["boxes_ol"].to(out_bbox.device)

            if tgt_ids.numel() == 0:
                # No targets: return empty match
                indices.append((torch.empty(0, dtype=torch.int64), torch.empty(0, dtype=torch.int64)))
                continue

            # classification cost: -P(class)
            cost_class = -out_prob[b][:, tgt_ids]  # [Q, Nt]

            # bbox L1 cost
            cost_bbox = torch.cdist(out_bbox[b], tgt_bbox, p=1)  # [Q, Nt]

            # giou cost
            out_xyxy = box_cxcywh_to_xyxy(out_bbox[b])
            tgt_xyxy = box_cxcywh_to_xyxy(tgt_bbox)
            cost_giou = -generalized_box_iou(out_xyxy, tgt_xyxy)  # [Q,Nt]

            # final cost matrix
            C = self.cost_bbox * cost_bbox + self.cost_class * cost_class + self.cost_giou * cost_giou
            C = C.cpu()

            # Hungarian algorithm
            i, j = linear_sum_assignment(C)
            indices.append((torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)))

        return indices


class SetCriterion(nn.Module):
    def __init__(self, num_classes, matcher, weight_dict, eos_coef=0.1):
        super().__init__()
        self.num_classes = num_classes
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef = eos_coef

        # Weight for "no-object" class
        empty_weight = torch.ones(num_classes + 1).to(device)
        empty_weight[-1] = eos_coef
        self.register_buffer("empty_weight", empty_weight)

    def loss_labels(self, outputs, targets, indices):
        src_logits = outputs["pred_logits"]  # [B, Q, K+1]
        bs, Q, _ = src_logits.shape

        # Default: all "no-object"
        target_classes = torch.full((bs, Q), self.num_classes, dtype=torch.int64, device=src_logits.device)

        # Assign matched classes
        for b, (src_idx, tgt_idx) in enumerate(indices):
            if src_idx.numel() == 0:
                continue
            target_classes[b, src_idx] = targets[b]["labels"][tgt_idx].to(src_logits.device)

        # Cross-entropy loss with weighting
        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, weight=self.empty_weight)
        return {"loss_ce": loss_ce}

    def loss_boxes(self, outputs, targets, indices):
        src_boxes = outputs["pred_boxes"]  # [B, Q, 4]
        bs, Q, _ = src_boxes.shape

        loss_bbox = torch.tensor(0., device=src_boxes.device)
        loss_giou = torch.tensor(0., device=src_boxes.device)

        # Compute bbox L1 and giou losses
        for b, (src_idx, tgt_idx) in enumerate(indices):
            if src_idx.numel() == 0:
                continue
            sb = src_boxes[b, src_idx]
            tb = targets[b]["boxes_ol"][tgt_idx].to(src_boxes.device)

            loss_bbox = loss_bbox + F.l1_loss(sb, tb, reduction="sum") / max(tb.shape[0], 1)

            giou = generalized_box_iou(box_cxcywh_to_xyxy(sb), box_cxcywh_to_xyxy(tb))
            loss_giou = loss_giou + (1 - giou.diag()).mean()

        return {"loss_bbox": loss_bbox, "loss_giou": loss_giou}

    def forward(self, outputs, targets):
        # Match predictions to targets
        indices = self.matcher(outputs, targets)
        losses = {}

        # Compute classification and bbox losses
        losses.update(self.loss_labels(outputs, targets, indices))
        losses.update(self.loss_boxes(outputs, targets, indices))

        # Weighted sum
        total = 0.0
        for k, v in losses.items():
            total += self.weight_dict.get(k, 1.0) * v
        losses["loss_total"] = total
        return losses

## ⚡ Setup for Training

- **Device**: CUDA if available, else CPU
- **Model**: Trans2RayModel with 15 classes, 100 queries, 256-dim features
- **Matcher**: HungarianMatcher (weights: class=1, bbox=5, giou=2)
- **Criterion**: SetCriterion with weighted losses
- **Optimizer**: AdamW (lr=1e-4, weight_decay=1e-4)
- **Scheduler**: StepLR (step every 20 epochs, gamma=0.1)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = Trans2RayModel(num_classes=15, num_queries=100, d_model=256, nhead=8, enc_layers=6, dec_layers=6).to(device)
matcher = HungarianMatcher(cost_class=1, cost_bbox=5, cost_giou=2)
criterion = SetCriterion(
    num_classes=15,
    matcher=matcher,
    weight_dict={"loss_ce": 1.0, "loss_bbox": 5.0, "loss_giou": 2.0},
    eos_coef=0.1
).to(device)
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4,
    weight_decay=1e-4
)

scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=20,
    gamma=0.1
)

In [ ]:
def save_checkpoint(model, optimizer, epoch, val_loss, filename="checkpoint.pth"):
    torch.save({
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "val_loss": val_loss,
    }, filename)


In [ ]:
def load_checkpoint(model, optimizer, filename="checkpoint.pth"):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch = checkpoint["epoch"]
    val_loss = checkpoint["val_loss"]
    return model, optimizer, epoch, val_loss


## Training Loop (Summary)

- Device: CUDA if available  
- Epochs: 50, grad clip: 0.1, log every 10 steps  
- Best model saved to `best_model.pth`  
- CSV log: `training_log.csv`

### Steps per Epoch:
1. **Train**: forward, loss, backprop, gradient clip, optimizer step, log  
2. **Validation**: eval mode, no grad, compute losses  
3. **Checkpoint**: save if val loss improves  
4. **Log**: append losses to CSV

In [ ]:
# -------------------------
# training config
# -------------------------
num_epochs = 50
grad_clip = 0.1
log_every = 10
save_path = "best_model.pth"
best_val_loss = float('inf')
log_file = "training_log.csv"

# ----------- create log file with header if not exists -----------
if not os.path.exists(log_file):
    with open(log_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            "epoch",
            "train_loss_total", "train_loss_ce", "train_loss_bbox", "train_loss_giou",
            "val_loss_total", "val_loss_ce", "val_loss_bbox", "val_loss_giou"
        ])

# -------------------------
# training loop
# -------------------------
for epoch in range(num_epochs):

    # =========================
    # TRAIN
    # =========================
    model.train()
    running_loss = 0.0
    running_ce = 0.0
    running_bbox = 0.0
    running_giou = 0.0

    pbar = tqdm(dl_train, desc=f"Epoch [{epoch+1}/{num_epochs}] Train")
    for step, (x_ol, x_sd, targets) in enumerate(pbar):

        x_ol = x_ol.to(device)
        x_sd = x_sd.to(device)

        outputs = model(x_ol, x_sd)
        losses = criterion(outputs, targets)
        loss = losses["loss_total"]

        optimizer.zero_grad()
        loss.backward()

        if grad_clip is not None:
            clip_grad_norm_(model.parameters(), grad_clip)

        optimizer.step()

        running_loss += loss.item()
        running_ce += losses["loss_ce"].item()
        running_bbox += losses["loss_bbox"].item()
        running_giou += losses["loss_giou"].item()

        if (step + 1) % log_every == 0:
            pbar.set_postfix({
                "L": f"{running_loss / (step + 1):.4f}",
                "CE": f"{running_ce / (step + 1):.4f}",
                "B": f"{running_bbox / (step + 1):.4f}",
                "G": f"{running_giou / (step + 1):.4f}",
                "lr": optimizer.param_groups[0]["lr"]
            })

    train_loss_total = running_loss / len(dl_train)
    train_loss_ce = running_ce / len(dl_train)
    train_loss_bbox = running_bbox / len(dl_train)
    train_loss_giou = running_giou / len(dl_train)

    # =========================
    # VALIDATION
    # =========================
    model.eval()
    val_loss = 0.0
    val_ce = 0.0
    val_bbox = 0.0
    val_giou = 0.0

    with torch.no_grad():
        for x_ol, x_sd, targets in tqdm(
            dl_val, desc=f"Epoch [{epoch+1}/{num_epochs}] Val"
        ):
            x_ol = x_ol.to(device)
            x_sd = x_sd.to(device)

            outputs = model(x_ol, x_sd)
            losses = criterion(outputs, targets)

            val_loss += losses["loss_total"].item()
            val_ce += losses["loss_ce"].item()
            val_bbox += losses["loss_bbox"].item()
            val_giou += losses["loss_giou"].item()

    val_loss_total = val_loss / len(dl_val)
    val_loss_ce = val_ce / len(dl_val)
    val_loss_bbox = val_bbox / len(dl_val)
    val_loss_giou = val_giou / len(dl_val)

    print(f"\nEpoch {epoch+1} Summary:")
    print(f"  Train | total: {train_loss_total:.4f} | ce: {train_loss_ce:.4f} | "
          f"bbox: {train_loss_bbox:.4f} | giou: {train_loss_giou:.4f}")
    print(f"  Val   | total: {val_loss_total:.4f} | ce: {val_loss_ce:.4f} | "
          f"bbox: {val_loss_bbox:.4f} | giou: {val_loss_giou:.4f}")

    # =========================
    # SAVE BEST MODEL
    # =========================
    if val_loss_total < best_val_loss:
        best_val_loss = val_loss_total
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved (epoch {epoch+1})")

    # =========================
    # CSV LOGGING
    # =========================
    with open(log_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            epoch + 1,
            train_loss_total, train_loss_ce, train_loss_bbox, train_loss_giou,
            val_loss_total, val_loss_ce, val_loss_bbox, val_loss_giou
        ])

Epoch [1/50] Train:  39%|███▉      | 2510/6400 [10:37<49:18,  1.31it/s, L=1.7847, CE=0.3378, B=0.1149, G=0.4362, lr=0.0001]  